In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
import scipy.integrate as integrate
import glob
import os
from multiprocessing import Pool, cpu_count
import multiprocessing as mp


from ipywidgets import VBox,HBox, Layout, interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import column, row, gridplot
from bokeh.models import BoxAnnotation, DataTable, TableColumn, PointDrawTool, HTMLTemplateFormatter

#from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter

from src.spectrum import spectrum
from src.spec_plot import spec_plot
from src.lcurve_plot import lcurve_plot
from src.mycolors import mycolors
from src.background_models import bg3D, bgFractal
from src.gradient_descent import gradient_descent
from src.error_plot import error_plot

output_notebook()

filelist = [os.path.basename(file) for file in glob.glob(os.path.join('data','raw','*.DTA'))]
file_picker=widgets.Dropdown(description= 'Choose File:', options= filelist)
file_picker

# Slice Averaging

In [ ]:
spec = spectrum(os.path.join('data','raw',file_picker.value))


#Plot definitions
xs= [spec.raw_time]*2
ys = [spec.real2D[0],spec.imaginary2D[0]]

spec2D = spec_plot(xs,ys,y_range = (spec.imaginary2D.min(),spec.real2D.max()*1.02), plot_height=300)

ys = [spec.real,spec.imaginary]
    
specavg = spec_plot(xs,ys, y_range = (spec.imaginary.min(),spec.real.max()*1.02), plot_height=300) 


#Widget Defintions

def on_button_clicked(val):
    if np.sum(spec.slice_mask)>1: #prevent deleting of all slices
        spec.delete_slice(myslider.value)
        myslider.options=valuelist[spec.slice_mask]
        myslider.value = myslider.options[-1]
        spec.avg_slices()
        update(myslider.value)
    
def update(index):
    for idx,line in enumerate(spec2D.renderers):
        line.data_source.data['y'] = [spec.real2D,spec.imaginary2D][idx][index]
        
    for idx,line in enumerate(specavg.renderers):
        line.data_source.data['y'] = [spec.real,spec.imaginary][idx]
        
    push_notebook(handle=sliceplots)
    return index

valuelist=np.arange(spec.real2D.shape[0])

myslider=widgets.SelectionSlider(description='value', value=valuelist[-1],options=valuelist)
deletebutton=widgets.Button(description='Delete Slice')
deletebutton.on_click(on_button_clicked)



#Render plots, widgets
sliceplots=show(row(spec2D,specavg),notebook_handle=True)
HBox([interactive(update,index=myslider),deletebutton])


# Phasing

In [ ]:
cutoffx=[spec.raw_time.max(),spec.raw_time.max()]
cutoffy=[-.1,1]

cutoffslider=widgets.SelectionSlider(description='Cutoff', value=spec.raw_time[-1], options=spec.raw_time)
signal=spec.real+1j*spec.imaginary
forphasing=signal[int(len(signal)/8):list(spec.raw_time).index(cutoffslider.value)]
spec.phase=optimize.fmin(lambda x:abs((forphasing*np.e**(1j*x)).imag.mean()),1,disp=False)[0]
phaseslider=widgets.FloatSlider(description='Phase', value=spec.phase, min=-np.pi, max=np.pi, step=0.001)


xs = [spec.raw_time]*2
ys = [np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase))),
      np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))]


phaseplot=spec_plot(xs,ys)
phaseplot.line(cutoffx,cutoffy, color=(255,0,0), line_width=1) #decorate plot with cutoff line
phaser=show(phaseplot, notebook_handle=True);


phasebutton=widgets.Button(description='Phase')
# kernelbutton=widgets.Button(description='Build Kernel')

def phase_button_clicked(val):
    forphasing=signal[int(len(signal)/8):list(spec.raw_time).index(cutoffslider.value)]
    spec.phase=optimize.fmin(lambda x:abs((forphasing*np.e**(1j*x)).imag.mean()),1,disp=False)[0]
    phaseslider.value=spec.phase
    
# def kernel_button_clicked(val):
    
#     spec.tmax = cutoffslider.value
#     spec.cutoff=cutoffslider.value # is cutoff needed (and not just as spec.tmax) ??
#     spec.build_kernel()
#     spec.real=np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
#     spec.imag=np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
    
   

    
def updatePhase(value):
    phaseplot.renderers[-3].data_source.data['y'] = np.real(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))
    phaseplot.renderers[-2].data_source.data['y'] = np.imag(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))
    phaseplot.renderers[-1].data_source.data['y'] = [np.min(np.imag(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))),1]
    push_notebook(handle=phaser)
    return value

def updateCutoff(value):
    phaseplot.renderers[-1].data_source.data['x'] = [value,value]
    push_notebook(handle=phaser)
    return value

#kernel,spec.t,spec.r,Lmatrix,spec.cutoff = 

phasebutton.on_click(phase_button_clicked)
# kernelbutton.on_click(kernel_button_clicked)
# HBox([interactive(updateCutoff,value=cutoffslider),interactive(updatePhase,value=phaseslider),phasebutton,kernelbutton])
HBox([interactive(updateCutoff,value=cutoffslider),interactive(updatePhase,value=phaseslider),phasebutton])

# Build Kernel, Set t=0 and Fit Initial Background

In [ ]:
#initial fit, finding spectrum start
#need to reimplement different types of background models

###originally in a build kernel button, but easy to forget to click
spec.tmax = cutoffslider.value
spec.cutoff=cutoffslider.value # is cutoff needed (and not just as spec.tmax) ??
spec.build_kernel()
spec.real=np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
spec.imag=np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
######


def gaussian(x,r,s,a):
    return a*np.exp(-((x-r)**2/(2*s**2)))

symmetry_point=np.argmax(spec.raw_time>=-spec.raw_time[0])+1 #find index of symmetry point about t=0,+1 for non inclusive
spec.zeropoint,__,spec.zeroamp = gaussfit = optimize.curve_fit(gaussian,
                                                               spec.raw_time[1:symmetry_point],
                                                               spec.real[1:symmetry_point])[0]
spec.bgmodel = bg3D
spec.update_ranges()
spec.fit_background(spec.bgmodel,spec.background_range)
spec.background_correct()

#zeropeak plot

xs = [spec.raw_time[1:symmetry_point]]*2
ys = [spec.real[1:symmetry_point],
      gaussian(spec.raw_time,*gaussfit)[1:symmetry_point]]

zeropointx = [spec.zeropoint]*2
zeropointy = [spec.real[1:symmetry_point].min(),1]

zeropeak = spec_plot(xs,ys, plot_height=300)
zeropeak.line(zeropointx, zeropointy, color=(0,255,0))
zeropeak.circle([spec.zeropoint], [spec.zeroamp], color=(255,0,0))

#bg fit plot

xs = [spec.raw_time]*2
ys = [spec.real,spec.background]

cutoffx=[spec.cutoff]*2
cutoffy=[spec.real.min(),1]

backgroundx=[spec.background_start]*2
backgroundy=[spec.real.min(),1]

bgsub=spec_plot(xs,ys,plot_height=300)
bgsub.line(cutoffx,cutoffy,color=(255,0,0))
bgsub.line(backgroundx,backgroundy,color=(0,0,255))

#waveform plot 
waveformplot=spec_plot(spec.t,spec.waveform,plot_height=300,y_range = (-.2,1.1))
waveformplot.line(spec.t,0,line_color = 'black', line_dash = 'dashed', line_width = 1)


processingplots = show(gridplot([[zeropeak,bgsub],[None,waveformplot]]),notebook_handle=True)

windowslider=widgets.IntSlider(description='Fit window', 
                               value=symmetry_point, min=4+np.argmax(spec.real==spec.real.max()), max=round(len(spec.raw_time)/30))
bgslider=widgets.SelectionSlider(description='Background Start', 
                                 value=spec.background_start, 
                                 options=spec.raw_time[4+np.argmax(spec.real==spec.real.max()):np.argmax(spec.raw_time==spec.cutoff)-20])
normalizer=widgets.Dropdown(description= 'V(t=0)', options=["Gaussian Fit",
                                   "Gaussian Center, Data Maximum Amp", 
                                   "Data Max Center, Data Maximum Amp"])

bgmodel_select = widgets.Dropdown(description= 'BG Model', options=["3D","Fractal"])



def updateBg(value): 
    spec.background_start = value
    spec.update_ranges()

    spec.fit_background(spec.bgmodel,spec.background_range)
    bgsub.renderers[-3].data_source.data['y'] = spec.background
    bgsub.renderers[-1].data_source.data['x']=[value]*2
    
    spec.background_correct()
    
    waveformplot.renderers[-2].data_source.data={'x':spec.t,'y':spec.waveform}
    
    push_notebook(handle=processingplots)


def updateWindow(value):
    spec.zeropoint,__,spec.zeroamp = gaussfit = optimize.curve_fit(gaussian,
                                                                   spec.raw_time[1:value],
                                                                   spec.real[1:value])[0]

    if normalizer.value == "Gaussian Center, Data Maximum Amp":
        spec.zeroamp = spec.real[abs(spec.raw_time-spec.zeropoint).argmin()]
    elif normalizer.value == "Data Max Center, Data Maximum Amp":
        spec.zeropoint = spec.raw_time[np.argmax(spec.real==spec.real.max())]
        spec.zeroamp = spec.real.max()
        
    spec.update_ranges()
    updateBg(bgslider.value)
    
    newx=spec.raw_time[1:value]
    newy=spec.real[1:value]
    fity=gaussian(newx,*gaussfit)
    plot_order = [[newx,newy],
                  [newx,fity],
                  [[spec.zeropoint]*2,[newy.min(),1]],
                  [[spec.zeropoint],[spec.zeroamp]]]
    
    for idx, line in enumerate(zeropeak.renderers):
        line.data_source.data = {'x': plot_order[idx][0], 'y': plot_order[idx][1]}

    
    push_notebook(handle=processingplots)

def updatebgModel(value):
    if bgmodel_select.value == '3D':
        spec.bgmodel = bg3D
    elif bgmodel_select.value == 'Fractal':
        spec.bgmodel = bgFractal
        
    updateBg(bgslider.value)
    

def updateNorm(value):
    normMethod = value
    updateWindow(windowslider.value)
    updateBg(bgslider.value)

VBox([HBox([interactive(updateWindow,value=windowslider),interactive(updateNorm,value=normalizer),interactive(updatebgModel,value=bgmodel_select)]),
HBox([interactive(updateBg,value=bgslider)])])

# L-Curve Generation

In [ ]:
spec.generate_lcurve()

#l-curve compoenents, initialize choice of alpha with LOOCV
residualnorm=np.linalg.norm(spec.tikhonovfits-spec.waveform,axis=1)**2
solutionnorm=np.linalg.norm(np.dot(spec.Lmatrix,spec.solutions.T).T,axis=1)**2
alpha_LOOCV_index=(residualnorm/(np.sum(1-spec.hMats,axis=1))**2).argmin()

#NOT SURE IM COMPUTING LOOCV CORRECTLY ABOVE, NEED TO GO RE-WORK


#l-curve
lcurve = lcurve_plot([residualnorm,solutionnorm],plot_height=300)
lcurve.circle([residualnorm[alpha_LOOCV_index]], [solutionnorm[alpha_LOOCV_index]],color=(255,0,0)) #LOOCV choice for alpha
lcurve.circle([residualnorm[alpha_LOOCV_index]], [solutionnorm[alpha_LOOCV_index]],color=(0,0,0)) #user defined choice for alpha

#waveform fit
xs = [spec.t]*2
ys = [spec.waveform,spec.tikhonovfits[alpha_LOOCV_index]]

tikresults=spec_plot(xs,ys,plot_height=300)
tikresults.line(spec.t,0,line_color = 'black', line_dash = 'dashed', line_width = 1)

#fit residual
tikresidual=spec_plot(spec.t,spec.waveform-spec.tikhonovfits[alpha_LOOCV_index],
                      plot_height=300,
                      y_range=([(spec.waveform-spec.tikhonovfits[alpha_LOOCV_index]).min(),
                                (spec.waveform-spec.tikhonovfits[alpha_LOOCV_index]).max()]))

#Distance distributions

xs = [spec.r]*2
ys = [spec.solutions[alpha_LOOCV_index]]*2
distances = spec_plot(xs,ys,plot_height = 300,colorlist = [(255,0,0),mycolors(0)])
distances.xaxis.axis_label="Distance (nm)"
# show the results


lcurveplots = show(gridplot([[lcurve,tikresidual],[tikresults,distances]]),notebook_handle=True)

def updateAlpha(value):
    spec.alpha=spec.alphas[value]
    tikresidual.renderers[-1].data_source.data['y']=spec.waveform-spec.tikhonovfits[value]
    tikresults.renderers[-2].data_source.data['y']=spec.tikhonovfits[value]
    lcurve.renderers[-1].data_source.data={'x':[residualnorm[value]],'y':[solutionnorm[value]]}
    distances.renderers[-1].data_source.data['y'] = spec.solutions[value]
    spec.solution = spec.solutions[value]
    spec.tikhonovfit = spec.tikhonovfits[value]

    push_notebook(handle=lcurveplots)

#slider for index of list of alphas, actual alpha value can be found with spec.alpha
alphaslider=widgets.IntSlider(description='Alpha Value', value=alpha_LOOCV_index, min=0, max=len(spec.alphas)-1)
HBox([interactive(updateAlpha,value=alphaslider)])

# Background Validation

In [ ]:
##BG VALIDATION
spec.validate_background(end_offset = 100)

from scipy.signal import argrelextrema
from scipy.stats import f as ftest

# Measure residual of fits rescaled to uncorrected bg spectra to normalize noise

amplitudes = [(1.0 - spec.backgrounds[i,spec.waveform_range][0])/(spec.backgrounds[i,spec.waveform_range][0]) 
              for i in range(len(spec.backgrounds))]

reconstructed = [(np.dot(spec.kernel,spec.validation_solutions[i])/np.dot(spec.kernel,spec.validation_solutions[i]).max())
                 *amplitudes[i]*spec.backgrounds[i,spec.waveform_range]+spec.backgrounds[i,spec.waveform_range] 
                 for i in range(len(spec.backgrounds))]

validation_RMSD = np.sqrt(np.mean((spec.real[spec.waveform_range] - reconstructed)**2, axis = 1))

# find local maxima, wellR and find wellL at the same RMSD value
maxima_index = argrelextrema(validation_RMSD, np.greater)[0][0]

#BUG HERE, this may fail!!
try:
    wellL = np.where(validation_RMSD[:maxima_index]>=validation_RMSD[maxima_index])[0][-1]
except IndexError:
    wellL = 0
wellR = maxima_index
RMSD_min_index = validation_RMSD[wellL:wellR].argmin()


xs = spec.raw_time[[np.argwhere(spec.raw_time == point)[0][0] for point in spec.background_fit_points]]
ys = validation_RMSD
rmsd_plot = spec_plot(xs,ys,plot_height = 300, y_range = (min(validation_RMSD[wellL:wellR])*.98,validation_RMSD[wellL]))

#my_DOF this is clearly incorrect, need to figure out DOF based on LOOCV but is good enought
#since the purpose is to determine the bottoms of the background validation well only.
my_DOF = len(spec.waveform) - 3 
# f_test = np.where(0.6 >= np.round(ftest.cdf(validation_RMSD[wellL:wellR]/np.median(validation_RMSD[wellL:wellR]),my_DOF,my_DOF),decimals = 2))[0]
f_test = np.where(0.6 >= np.round(ftest.cdf(validation_RMSD/validation_RMSD[RMSD_min_index],my_DOF,my_DOF),decimals = 2))[0]
try:
    f_test = f_test[:np.where(1<(f_test[1:]-f_test[:-1]))[0][0]+1] #find first discontinuity in ftest (edge of well)
except IndexError:
    f_test = f_test
    


rmsd_plot.circle(xs,ys)

rmsd_plot.line(xs[f_test],validation_RMSD[f_test],color = (0,0,0), line_width=3)
rmsd_plot.circle([xs[RMSD_min_index]],[validation_RMSD[RMSD_min_index]], color = (255,0,0))
rmsd_plot.circle([xs[RMSD_min_index]],[validation_RMSD[RMSD_min_index]], color = (0,255,0))

rmsd_plot.xaxis.axis_label="Background Fit Start (µs)"
rmsd_plot.yaxis.axis_label_text_font='helvetica'
rmsd_plot.yaxis.axis_label_text_font_size='14pt'


rmsd_plot.yaxis.axis_label="Spectrum Fit RMSD"

backgrounds_plot=spec_plot([spec.raw_time]*3, #x-values
                      [spec.real,spec.background,spec.backgrounds[RMSD_min_index]], #y-values
                      plot_height=300)

spectrums_plot = spec_plot([spec.t]*2,
                                 [spec.waveforms[RMSD_min_index],spec.validation_tikhonovfits[RMSD_min_index]],
                                 plot_height = 300, y_range = (np.min(spec.waveforms),1.1)) #This is a DDT Lab joke
spectrums_plot.line(spec.t,0,line_color = 'black', line_dash = 'dashed', line_width = 1)

distances_plot = spec_plot([spec.r]*2,
                                   [spec.validation_solutions[RMSD_min_index],spec.validation_solutions[RMSD_min_index]],
                                   y_range = (0,np.max(spec.validation_solutions)), 
                                   plot_height = 300)
distances_plot.varea(x=spec.r,
        y1=np.quantile(spec.validation_solutions,0, axis=0),
        y2=np.quantile(spec.validation_solutions,1, axis=0), fill_alpha = 0.2, level = 'underlay')

distances_plot.xaxis.axis_label="Distance (nm)"


validation_plots = show(gridplot([[rmsd_plot,backgrounds_plot],[spectrums_plot,distances_plot]]), notebook_handle = True)

def updateRegion(end_points):
    new_range = range(*end_points)
    
    if not new_range:
        new_range = [end_points[0]] #happens if the range slider handles are on top of eachother
        
    RMSD_min_index = validation_RMSD[new_range].argmin()
    rmsd_plot.renderers[-2].data_source.data={'x':[xs[RMSD_min_index]],'y': [validation_RMSD[RMSD_min_index]]}
    rmsd_plot.renderers[-3].data_source.data={'x':xs[new_range],'y': validation_RMSD[new_range]}
    rmsd_plot.y_range.end = validation_RMSD[new_range].max()*1.02
    spectrums_plot.y_range.start = np.min(spec.waveforms[new_range]) #this throws an error if the range is 0
    #try:
    distances_plot.renderers[-1].data_source.data = {'x': spec.r,
                                                     'y1': np.quantile(spec.validation_solutions[new_range], 0, axis=0),
                                                     'y2': np.quantile(spec.validation_solutions[new_range], 1, axis=0)}
    distances_plot.renderers[0].data_source.data['y'] = spec.validation_solutions[RMSD_min_index]
    
    bgslider.options = new_range
#     except IndexError:
#         print('Zero width range, distances envelope not updated')
#         bgslider.options = [end_points[0]]
#         #dont update if range is 0 width
    
    push_notebook(handle = validation_plots)
    
def updateSelection(value):
    rmsd_plot.renderers[-1].data_source.data={'x':[xs[value]],'y': [validation_RMSD[value]]}
    backgrounds_plot.renderers[-1].data_source.data['y'] = spec.backgrounds[value]
    spectrums_plot.renderers[-3].data_source.data['y'] = spec.waveforms[value]
    spectrums_plot.renderers[-2].data_source.data['y'] = spec.validation_tikhonovfits[value]
    distances_plot.renderers[-2].data_source.data['y'] = spec.validation_solutions[value]
    
    spec.background_start = spec.background_fit_points[bgslider.value]
    spec.waveform = spec.waveforms[bgslider.value]
    spec.tikhonovfit = spec.validation_tikhonovfits[bgslider.value]
    spec.solution = spec.validation_solutions[bgslider.value]
    
    push_notebook(handle = validation_plots)

rmsdSlider = widgets.IntRangeSlider(description = 'Backgrounds:', value =list(f_test[[0,-1]]), min = 0, max = validation_RMSD.shape[0])
bgslider = widgets.SelectionSlider(description = 'Selection:', value = RMSD_min_index, options = range(rmsdSlider.value[0],rmsdSlider.value[1]+1))

VBox([interactive(updateRegion,end_points=rmsdSlider),interactive(updateSelection,value=bgslider)])




# Background Supplementation

In [ ]:
width_cutoff = 4*(spec.t[-1]/2)**(1/3)
distance_cutoff = 5*(spec.t[-1]/2)**(1/3)
#Jeschke, G. (2012). DEER distance measurements on proteins. Annual Review of Physical Chemistry, 63, 419–446.
instability_point_index = abs(spec.r-(width_cutoff+(distance_cutoff-width_cutoff)/2)).argmin()
instability_point_start = spec.r[instability_point_index]
distance_mask = spec.r*0
distance_mask[instability_point_index:] = 1

spec.unstable_waveform = np.dot(spec.kernel, spec.solution*distance_mask)
spec.stable_waveform = (spec.waveform - spec.unstable_waveform)/(1.0 - spec.unstable_waveform.max())
spec.stable_solution = spec.waveform_regularize(spec.stable_waveform)
#below needs to be revisited, seems like solution shouldnt be modified after the fact
spec.stable_solution = spec.stable_solution/integrate.trapz(spec.stable_solution)
spec.stable_fit = np.dot(spec.kernel,spec.stable_solution)
spec.stable_fit = spec.stable_fit/spec.stable_fit[0]


instability_plot = spec_plot(spec.r,spec.solution,y_range = (0,spec.solution.max()*1.1), plot_height = 300)
instability_plot.line([instability_point_start]*2,[0,spec.solution.max()*1.1],color = (0,0,0), line_dash = 'dashed')
instability_plot.varea(x=spec.r,
                       y1=np.quantile(spec.validation_solutions[range(*rmsdSlider.value)],0, axis=0),
                       y2=np.quantile(spec.validation_solutions[range(*rmsdSlider.value)],1, axis=0), 
                       fill_alpha = 0.2, level = 'underlay')

green_box = BoxAnnotation(left=0, right=width_cutoff, fill_color='green', fill_alpha=0.1)
yellow_box = BoxAnnotation(left=width_cutoff, right=distance_cutoff, fill_color='yellow', fill_alpha=0.1)
red_box = BoxAnnotation(left=width_cutoff, right=10, fill_color='red', fill_alpha=0.1)
instability_plot.add_layout(green_box)
instability_plot.add_layout(yellow_box)
instability_plot.add_layout(red_box)
instability_plot.xaxis.axis_label = "Distance (nm)"

filtered_distances = spec_plot([spec.r,spec.r],
                               [spec.solution/integrate.trapz(spec.solution-spec.solution*distance_mask),spec.stable_solution],y_range = (-.001,spec.solution.max()*1.1), plot_height = 300)
filtered_distances.xaxis.axis_label = "Distance (nm)"

instability_cutoff = show(row(instability_plot,filtered_distances), notebook_handle = True)

def setCutoff(distance):
    instability_plot.renderers[-2].data_source.data['x'] = [distance,distance]
    
    cutoff_index = np.where(spec.r == distance)[0][0]
    distance_mask = spec.r*0
    distance_mask[cutoff_index:] = 1


    spec.unstable_waveform = np.dot(spec.kernel, spec.solution*distance_mask)
    spec.stable_waveform = (spec.waveform - spec.unstable_waveform)/(1.0 - spec.unstable_waveform.max())
    #below needs to be revisited, seems like solution shouldnt be modified after the fact
    spec.stable_solution = spec.waveform_regularize(spec.stable_waveform)
    spec.stable_solution = spec.stable_solution/integrate.trapz(spec.stable_solution)
    spec.stable_fit = np.dot(spec.kernel,spec.stable_solution)
    spec.stable_fit = spec.stable_fit/spec.stable_fit[0]
    
    filtered_distances.renderers[-1].data_source.data['y'] = spec.stable_solution
    filtered_distances.renderers[-2].data_source.data['y'] = spec.solution/integrate.trapz(spec.solution-spec.solution*distance_mask)
    
    push_notebook(handle = instability_cutoff)


#Does not currently update in real time, need to find a way to decouple with calculation of new distance distribution
instability_slider = widgets.SelectionSlider(description = 'Instability Point:', value = instability_point_start, options = spec.r,continuous_update=True)

HBox([interactive(setCutoff,distance = instability_slider)])

# Gaussian Modeling

In [ ]:
from bokeh.palettes import Category10_10 as Cat10

#Gaussian modeling utility
#create spec.gaussian_models
ngauss = 5
spec.gaussian_model_init(ngauss)
#flags for fitting and optimization
fitted = np.zeros(ngauss,dtype=int)
optimized = np.zeros(ngauss,dtype=int)
bics = np.ones(ngauss)
model_rmsds = np.zeros(ngauss)

def bic(data,fit,n_model_params):
    k = n_model_params+1
    n = len(data)
    return n*np.log(np.linalg.norm(data-fit)**2/n)+k*np.log(n)



model_plot = spec_plot(spec.r,spec.stable_solution, x_range = (1,10.1),plot_height = 300)
model_plot.xaxis.axis_label="Distance (nm)"
#initialize points on which to seed gaussian fits
for model in range(ngauss):
    centers = spec.gaussian_model(model)[:,0]
    amps = spec.stable_solution[np.argwhere([spec.r == item for item in centers])[:,1]]
    model_plot.circle(centers,amps,visible = False)
model_plot.renderers[1].visible = True 

#initialize eventual gaussian fit plots
for model in range(ngauss):
    model_plot.line(spec.r,spec.r*0, color = 'orange',visible = False)

stable_rmsd = np.sqrt(np.mean((spec.stable_waveform-spec.stable_fit)**2))*10**2

model_rmsd_plot = spec_plot(np.linspace(0,ngauss+1),np.array([stable_rmsd]*len(np.linspace(0,ngauss+1))),y_range = (0,1.5*stable_rmsd),plot_height = 300)
model_rmsd_plot.xaxis.axis_label="Number of Gaussians"
model_rmsd_plot.renderers[0].glyph.line_dash = 'dashed'
model_rmsd_plot.renderers[0].glyph.line_color = 'black'
model_rmsd_plot.line(np.arange(ngauss)[fitted==1]+1,model_rmsds[fitted==1], line_color = 'black', visible = False)
model_rmsd_plot.circle(np.arange(ngauss)[fitted==1]+1,model_rmsds[fitted==1], visible = True)
model_rmsd_plot.circle([0],[0], fill_color="white", size = 10, color = 'red', fill_alpha = 0,visible = False)

model_waveform_plot = spec_plot([spec.t]*3,
                                 [spec.stable_waveform,spec.stable_fit, spec.stable_fit*0],
                                 plot_height = 300, y_range = (np.min(spec.stable_waveform),1.1))
model_waveform_plot.renderers[0].glyph.line_color = 'gray'
model_waveform_plot.renderers[0].glyph.line_alpha = 0.7
model_waveform_plot.renderers[1].glyph.line_color = Cat10[0]
model_waveform_plot.renderers[2].glyph.line_color = Cat10[1]

model_residual_plot = spec_plot([spec.t]*2,[spec.stable_waveform-spec.stable_fit]*2,plot_height = 300)
    
gaussian_modeling = show(gridplot([[model_plot,model_rmsd_plot],[model_waveform_plot,model_residual_plot]]), notebook_handle=True)

def update_gaussians(model,ith_gaussian, value):
    spec.gaussian_model(model)[ith_gaussian,0] = value
    move_points(model)

def move_points(model, update_sliders = False):
    centers = spec.gaussian_model(model)[:,0]
    #centers may be due to fit, so finde nearest for spec.r to satisfy slider
    nearest_centers = spec.r[np.argmin([abs(spec.r-item) for item in centers], axis =1)]
    amps = spec.stable_solution[np.argwhere([spec.r == item for item in nearest_centers])[:,1]]
    model_plot.renderers[model+1].data_source.data = {'x': centers, 'y': amps}
    
    if update_sliders:
        #update sliders
        for slider_no, center in enumerate(nearest_centers):
            tabs.children[model].children[slider_no].children[0].value = center
    
    push_notebook(handle = gaussian_modeling)
    
def update_figure(widget):
    #Show only points corresponding to the selected tab, only gets called once tab is clicked
    tab_idx = widget['new']
    for idx in range(1,len(model_plot.renderers)):
        model_plot.renderers[idx].visible = False
    model_plot.renderers[tab_idx+1].visible = True
    
    if fitted[tab_idx]:
        model_plot.renderers[tab_idx+1+ngauss].visible = True

        gaussian_waveform = spec.model_waveform(gaussians,spec.gaussian_model(tab_idx))
        model_waveform_plot.renderers[2].data_source.data['y'] = gaussian_waveform
        model_residual_plot.renderers[1].data_source.data['y'] = spec.stable_waveform-gaussian_waveform
    
    push_notebook(handle = gaussian_modeling)


    
def gaussians(x,coeffs):
    #coeffs in form of [[ngauss,3]] and component order r,s,a
    peaks = (coeffs[:,2]*np.exp(-((x[:,np.newaxis]-coeffs[:,0])**2/(2*coeffs[:,1]**2)))).T
    return np.sum(peaks,axis = 0)

def curve_fit_gaussians(x,*coeffs):
    this_is_dumb = np.reshape(coeffs, (len(coeffs)//3,3))
    return gaussians(x,this_is_dumb)

def fit_gaussians(val):
    model = tabs.selected_index
    fitted[model] = 1
    optimized[model] = 0
    
    initial_con = spec.gaussian_model(model).ravel()
    #reset widths,amps so not trapped in bad fit
    initial_con[1::3] = np.zeros(model+1)+.1
    initial_con[2::3] = np.zeros(model+1)+.1
    
    low_bounds = np.array([[-1.2*value,-np.inf,-np.inf]for value in spec.gaussian_model(model)[:,0]]).flatten()
    high_bounds = low_bounds*-1
    
    fit_coeff = optimize.curve_fit(curve_fit_gaussians,
                                   spec.r,
                                   spec.stable_solution, 
                                   p0 = initial_con,
                                   bounds = (low_bounds,high_bounds),
                                   method = 'trf')[0].reshape(model+1,3)
    #update gaussian_models
    spec.gaussian_model(model)[:] = abs(fit_coeff)
    move_points(model, update_sliders = True)
    draw_gaussians(model)
    
def draw_gaussians(model):
    fit_curve = gaussians(spec.r,spec.gaussian_model(model))
    fit_curve = fit_curve/integrate.trapz(fit_curve)
   
    model_plot.renderers[model+1+ngauss].data_source.data = {'x': spec.r,'y': fit_curve}
    model_plot.renderers[model+1+ngauss].visible = True
    
    gaussian_waveform = spec.model_waveform(gaussians,spec.gaussian_model(model))
    
    bics[model] = bic(spec.stable_waveform,gaussian_waveform,(model+1)*3)
    
    bics_min = np.argmin(bics)
    spec.best_model_id = bics_min
    
    model_rmsds[model] = np.sqrt(np.mean((spec.stable_waveform-gaussian_waveform)**2))*10**2
    
    model_rmsd_plot.renderers[1].data_source.data = {'x': np.arange(ngauss)[fitted==1]+1 ,'y': model_rmsds[fitted==1]}
    model_rmsd_plot.renderers[2].data_source.data = {'x': np.arange(ngauss)[fitted==1]+1 ,'y': model_rmsds[fitted==1]}
    model_rmsd_plot.renderers[3].data_source.data = {'x': [np.arange(ngauss)[bics_min]+1] , 'y': [model_rmsds[bics_min]]}
    model_rmsd_plot.renderers[3].visible = True
    model_rmsd_plot.y_range.end = model_rmsds.max()*1.1
    
    model_waveform_plot.renderers[2].data_source.data['y'] = gaussian_waveform
    model_residual_plot.renderers[1].data_source.data['y'] = spec.stable_waveform-gaussian_waveform
    
    
    if np.sum(fitted)>=2:
        model_rmsd_plot.renderers[1].visible = True

    push_notebook(handle = gaussian_modeling)



def optimize_gaussians(val):
    
    model_func = lambda x: spec.model_waveform(gaussians,x)
    
    for model in np.where(optimized ^ fitted)[0]:
        tabs.selected_index = model
        spec.gaussian_model(model)[:] = gradient_descent(spec.stable_waveform,model_func,spec.gaussian_model(model))
        move_points(model, update_sliders = True)
        draw_gaussians(model)

        optimized[model] = 1

       
    
### Widget Initialization###
children = [VBox([interactive(update_gaussians, model = fixed(model), 
                              ith_gaussian = fixed(ith_gaussian), 
                              value = widgets.SelectionSlider(value = spec.gaussian_model(model)[ith_gaussian,0], 
                                                              description = str(ith_gaussian+1)+'G', 
                                                              options = spec.r)) 
                  for ith_gaussian in range(model+1)]) 
            for model in range(ngauss)]

tabs = widgets.Tab()
tabs.children = children
for model in range(ngauss):
    tabs.set_title(model, str(model+1)+'G')
    
tabs.observe(update_figure, names = 'selected_index')

fit_gauss_button=widgets.Button(description='Initialize Gaussian(s)')
optimize_gauss_button=widgets.Button(description='Fit Waveform')

fit_gauss_button.on_click(fit_gaussians)
optimize_gauss_button.on_click(optimize_gaussians)

HBox([tabs,VBox([fit_gauss_button,optimize_gauss_button])])



# Monte Carlo Error Surface

In [ ]:

#initial plot is determined by BIC minimum (red circle in plots above)
initial_iterations = 5000
optimized_models = np.argwhere(optimized).flatten()
deltar = spec.r[1] - spec.r[0]

#weight search space of r and s higher for small populations, giving larger step size
rs_weights = {optimized_model: np.round(spec.gaussian_model(optimized_model)[:,2].max()/spec.gaussian_model(optimized_model)[:,2]) 
                  for optimized_model in optimized_models}
step_weights = {optimized_model: np.hstack((np.tile(rs_weights[optimized_model][:,None],(1,2))*deltar,np.ones((optimized_model+1,1))*.01))
                                 for optimized_model in optimized_models}

spec.monte_carlo_error(data = spec.stable_waveform, 
                       origin = spec.gaussian_model(spec.best_model_id), 
                       weights = step_weights[spec.best_model_id],
                       iterations = initial_iterations)

#Properly normalize, convert to mole fractions
spec.norm_constants = np.array([integrate.trapz(gaussians(spec.r,coeff),spec.r) for coeff in spec.model_landscape])
#spec.model_landscape[:,:,2] = np.sqrt(2*np.pi)*spec.model_landscape[:,:,1]*spec.model_landscape[:,:,2]/norm_constants[:,None]

def reset_weights(val):
    
    for i,kids in enumerate(weights_tabs.children):
        for j,kid in enumerate(kids.children):
            kid.value = rs_weights[i][j]
    


def update_error_plot(val):
    
    #weight search space of r and s higher for small populations, giving larger step sized
    new_rs_weights = np.array([child.value for child in weights_tabs.children[weights_tabs.selected_index].children])
    new_step_weights = np.hstack((np.tile(new_rs_weights[:,np.newaxis],(1,2))*deltar,np.ones((weights_tabs.selected_index+1,1))*.01))

    spec.monte_carlo_error(data = spec.stable_waveform, 
                           origin = spec.gaussian_model(weights_tabs.selected_index), 
                           weights = new_step_weights,
                           iterations = iteration_widget.value)
    
    #Properly normalize, convert to mole fractions
    spec.norm_constants = np.array([integrate.trapz(gaussians(spec.r,coeff),spec.r) for coeff in spec.model_landscape])
    #spec.model_landscape[:,:,2] = np.sqrt(2*np.pi)*spec.model_landscape[:,:,1]*spec.model_landscape[:,:,2]/norm_constants[:,None]

    surface.children, slicegrid.children = error_plot(spec.model_landscape.copy(),spec.landscape_statistics,refresh = True, norm_constants = spec.norm_constants)
    push_notebook(handle = handle)
    
    

plotbutton=widgets.Button(description='Update Plot')
plotbutton.on_click(update_error_plot)
resetbutton = widgets.Button(description = 'Reset Weights')
resetbutton.on_click(reset_weights)
    
handle, surface, slicegrid = error_plot(spec.model_landscape.copy(),spec.landscape_statistics, norm_constants = spec.norm_constants)

### Widget Initialization###
iteration_widget = widgets.IntText(value = initial_iterations, description='Iterations:')
children = [VBox([widgets.IntText(value = value, description='Pop '+str(idx+1))
                  for idx,value in enumerate(rs_weights[model])]) 
            for model in optimized_models]

weights_tabs = widgets.Tab()
weights_tabs.children = children
for model in optimized_models:
    weights_tabs.set_title(model, str(model+1)+'G')
weights_tabs.selected_index = spec.best_model_id
    
HBox([weights_tabs,VBox([plotbutton,resetbutton,iteration_widget])])


# Fit Menagerie

In [ ]:
### Show best fit from montecarlo search and 95% confidence range

from bokeh.palettes import Colorblind6 as palette

spec.best_model = spec.model_landscape[np.argmin(spec.landscape_statistics)]
spec.best_model_fit = spec.model_waveform(gaussians,spec.best_model)
spec.best_model_solution = gaussians(spec.r,spec.best_model)

norm_constant = integrate.trapz(spec.best_model_solution,spec.r)
spec.best_model_solution = spec.best_model_solution/norm_constant

model_variability = np.array([spec.model_waveform(gaussians,model) 
                              for model in  spec.model_landscape[spec.landscape_statistics<.95]])



#Show best model and 95% bands
monte_spectra_plot = spec_plot(plot_height = 300)
monte_spectra_plot.line(spec.t,spec.stable_waveform,line_color='gray')
monte_spectra_plot.line(spec.t,spec.best_model_fit,line_color = palette[0])
monte_spectra_plot.varea(x=spec.t,
                       y1=np.quantile(model_variability,0, axis=0),
                       y2=np.quantile(model_variability,1, axis=0), 
                       fill_alpha = 0.5, level = 'underlay', fill_color = palette[5])

#Show gaussian model and comprising populations
sub_gauss_plot = spec_plot(plot_height = 300)
sub_gauss_plot.xaxis.axis_label = 'Distance (nm)'
sub_gauss_plot.line(spec.r,spec.stable_solution/integrate.trapz(spec.stable_solution,spec.r),line_color = 'gray')

for pop in range(spec.best_model.shape[0]):
    
    sub_gauss_plot.line(spec.r,gaussians(spec.r,spec.best_model[pop,None])/norm_constant)
sub_gauss_plot.line(spec.r,gaussians(spec.r,spec.best_model)/norm_constant,line_color = 'orange')

show(row(monte_spectra_plot,sub_gauss_plot))

# Output Results

In [ ]:
to_save = ['full_t','raw_time', 'real', 'imag', 'background', 't','waveform','tikhonovfit',
           'stable_waveform','unstable_waveform', 'stable_fit','best_model_fit',
           'r','solution', 'stable_solution', 'best_model_solution']
           #'zeroamp', 'zeropoint','normamp','tmax','phase','alpha','background_start']

filename = os.path.basename(spec.path).split('.')[0]
output = pd.DataFrame(columns = to_save)

specvars = vars(spec)

output = pd.DataFrame(columns = to_save)
for item in to_save:
    output[item] = pd.Series(specvars[item])
#Add renormalized Tikhonov
output['Rescaled Tik'] = pd.Series(spec.stable_solution/integrate.trapz(spec.stable_solution,spec.r))
    
#Add sub populations to the end
for pop in range(spec.best_model.shape[0]):
    output['Population: ' + str(pop+1)] = pd.Series(gaussians(spec.r,spec.best_model[pop,None])/norm_constant)
    

#convert to mole fractions
spec.model_landscape_scaled = spec.model_landscape.copy()
spec.model_landscape_scaled[:,:,2] = np.sqrt(2*np.pi)*spec.model_landscape_scaled[:,:,1]*spec.model_landscape_scaled[:,:,2]/spec.norm_constants[:,None]
spec.best_model_scaled = spec.model_landscape_scaled[np.argmin(spec.landscape_statistics)]

thresholds = [.6,.75,.95]
for thresh in thresholds:
    gaussian_output = pd.DataFrame(index = list(range(1,spec.best_model.shape[0]+1)),columns = ['[r (nm), lb, ub]', '[σ (nm), lb, ub]', '[Mol Frac, lb, ub]'])
    lower_bound = spec.model_landscape_scaled[spec.landscape_statistics<thresh]
    upper_bound = spec.model_landscape_scaled[spec.landscape_statistics<thresh]
    for i in range(spec.best_model.shape[0]):
        for j in range(3):   
            gaussian_output.iloc[i,j] =np.round([spec.best_model_scaled[i,j],lower_bound[:,i,j].min(),upper_bound[:,i,j].max()],decimals = 3)
    gaussian_output.to_csv(os.path.join('results',filename+'_'+str(int(thresh*100))+'cutoff_'+'model.csv'))

        
output.to_csv(os.path.join('results',filename+'_results.csv'))
